In [1]:
import time
import math
import numpy as np
import numba as nb
from numba import cuda
import torch

In [2]:
nb.cuda.detect()

Found 1 CUDA devices
id 0    b'NVIDIA GeForce RTX 2070 SUPER'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-d76dbc1d-cbf5-d7ba-7ff2-b858d30bc8f2
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [3]:
sizes = [256, 1024, 4096]
iterations = 10

## numba (CUDA)　での行列積速度

In [4]:
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 32

In [4]:
@nb.cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = nb.cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [6]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

/home/kensho/.pyenv/versions/3.8.12/lib/python3.8/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (64) < 2 * SM count (80) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/kensho/.pyenv/versions/3.8.12/lib/python3.8/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (64) < 2 * SM count (80) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


行列サイズ = 256
処理時間 : memcpy = 0.006049704551696777
処理時間 : kernel = 0.0007893562316894532
処理時間 : total  = 0.006937479972839356
行列サイズ = 1024
処理時間 : memcpy = 0.0021799802780151367
処理時間 : kernel = 0.03197135925292969
処理時間 : total  = 0.034659957885742186
行列サイズ = 4096
処理時間 : memcpy = 0.021010565757751464
処理時間 : kernel = 1.7810150146484376
処理時間 : total  = 1.812699317932129


In [9]:
@nb.cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)
    sB = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)

    x, y = nb.cuda.grid(2)

    tx = nb.cuda.threadIdx.x
    ty = nb.cuda.threadIdx.y
    bpg = nb.cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        nb.cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        nb.cuda.syncthreads()

    C[x, y] = tmp

In [8]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        fast_matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

/home/kensho/.pyenv/versions/3.8.12/lib/python3.8/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (64) < 2 * SM count (80) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


行列サイズ = 256
処理時間 : memcpy = 0.0008071184158325196
処理時間 : kernel = 0.000883793830871582
処理時間 : total  = 0.0017897844314575194
行列サイズ = 1024
処理時間 : memcpy = 0.0021622419357299806
処理時間 : kernel = 0.03261787891387939
処理時間 : total  = 0.035291433334350586
行列サイズ = 4096
処理時間 : memcpy = 0.020765161514282225
処理時間 : kernel = 1.9950968503952027
処理時間 : total  = 2.026572012901306


## numba ブロックあたりのスレッド数を変えた場合

In [4]:
TPB = 16

In [5]:
@nb.cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = nb.cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [6]:
@nb.cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)
    sB = nb.cuda.shared.array(shape=(TPB, TPB), dtype=nb.float32)

    x, y = nb.cuda.grid(2)

    tx = nb.cuda.threadIdx.x
    ty = nb.cuda.threadIdx.y
    bpg = nb.cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        nb.cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        nb.cuda.syncthreads()

    C[x, y] = tmp

In [7]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB, 1)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]), 1)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

行列サイズ = 256
処理時間 : memcpy = 0.005876660346984863
処理時間 : kernel = 0.00043044090270996096
処理時間 : total  = 0.006404805183410645
行列サイズ = 1024
処理時間 : memcpy = 0.0021755218505859373
処理時間 : kernel = 0.0193359375
処理時間 : total  = 0.022016334533691406
行列サイズ = 4096
処理時間 : memcpy = 0.021129441261291505
処理時間 : kernel = 0.9916394948959351
処理時間 : total  = 1.0234895944595337


In [8]:
for size in sizes:
    x = np.random.randn(size, size).astype(np.float32)
    y = np.random.randn(size, size).astype(np.float32)
    out = np.zeros((size, size)).astype(np.float32)

    threads = (TPB, TPB)
    blocks = (math.ceil(size / threads[0]), math.ceil(size / threads[1]))

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        start = time.time()

        d_x = nb.cuda.to_device(x)
        d_y = nb.cuda.to_device(y)
        d_o = nb.cuda.to_device(out)

        # 時間計測 (memcpy)
        nb.cuda.synchronize()
        memcpy_end = time.time()
        if i != 0:
            d_sec_memcpy += (memcpy_end - start)


        fast_matmul[blocks, threads](d_x, d_y, d_o)

        # 時間計測 (kernel)
        nb.cuda.synchronize()
        kernel_end = time.time()
        if i != 0:
            d_sec_kernel += (kernel_end - memcpy_end)


        out = d_o.copy_to_host()

        # 時間計測 (total)
        nb.cuda.synchronize()
        end = time.time()
        if i != 0:
            d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

行列サイズ = 256
処理時間 : memcpy = 0.000793290138244629
処理時間 : kernel = 0.00037403106689453124
処理時間 : total  = 0.0012653589248657227
行列サイズ = 1024
処理時間 : memcpy = 0.0021138906478881834
処理時間 : kernel = 0.015891551971435547
処理時間 : total  = 0.01850621700286865
行列サイズ = 4096
処理時間 : memcpy = 0.02103612422943115
処理時間 : kernel = 1.0008277654647828
処理時間 : total  = 1.0325467109680175


## pytorchでの行列積速度

In [10]:
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

device = torch.device(0)

In [11]:
for size in sizes:
    x = torch.randn((size, size), dtype=torch.float32, requires_grad=False)
    y = torch.randn((size, size), dtype=torch.float32, requires_grad=False)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        with torch.no_grad():
            start = time.time()

            d_x = x.to(device)
            d_y = y.to(device)

            # 時間計測 (memcpy)
            torch.cuda.synchronize()
            memcpy_end = time.time()
            if i != 0:
                d_sec_memcpy += (memcpy_end - start)


            d_out = torch.matmul(d_x, d_y)

            # 時間計測 (kernel)
            torch.cuda.synchronize()
            kernel_end = time.time()
            if i != 0:
                d_sec_kernel += (kernel_end - memcpy_end)


            out = d_out.to("cpu")

            # 時間計測 (total)
            torch.cuda.synchronize()
            end = time.time()
            if i != 0:
                d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

行列サイズ = 256
処理時間 : memcpy = 9.646415710449219e-05
処理時間 : kernel = 3.139972686767578e-05
処理時間 : total  = 0.00019218921661376954
行列サイズ = 1024
処理時間 : memcpy = 0.0008085727691650391
処理時間 : kernel = 0.00033447742462158205
処理時間 : total  = 0.002263975143432617
行列サイズ = 4096
処理時間 : memcpy = 0.011610937118530274
処理時間 : kernel = 0.015471220016479492
処理時間 : total  = 0.04687211513519287


In [12]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [13]:
for size in sizes:
    x = torch.randn((size, size), dtype=torch.float32, requires_grad=False)
    y = torch.randn((size, size), dtype=torch.float32, requires_grad=False)

    d_sec_memcpy = 0
    d_sec_kernel = 0
    d_sec_total = 0
    for i in range(iterations+1):
        with torch.no_grad():
            start = time.time()

            d_x = x.to(device)
            d_y = y.to(device)

            # 時間計測 (memcpy)
            torch.cuda.synchronize()
            memcpy_end = time.time()
            if i != 0:
                d_sec_memcpy += (memcpy_end - start)


            d_out = torch.matmul(d_x, d_y)

            # 時間計測 (kernel)
            torch.cuda.synchronize()
            kernel_end = time.time()
            if i != 0:
                d_sec_kernel += (kernel_end - memcpy_end)


            out = d_out.to("cpu")

            # 時間計測 (total)
            torch.cuda.synchronize()
            end = time.time()
            if i != 0:
                d_sec_total += (end - start)


    d_sec_memcpy /= iterations
    d_sec_kernel /= iterations
    d_sec_total /= iterations
    print(f"行列サイズ = {size}")
    print(f"処理時間 : memcpy = {d_sec_memcpy}")
    print(f"処理時間 : kernel = {d_sec_kernel}")
    print(f"処理時間 : total  = {d_sec_total}")

行列サイズ = 256
処理時間 : memcpy = 9.932518005371094e-05
処理時間 : kernel = 3.235340118408203e-05
処理時間 : total  = 0.00019829273223876954
行列サイズ = 1024
処理時間 : memcpy = 0.0009094476699829102
処理時間 : kernel = 0.0003421783447265625
処理時間 : total  = 0.002171969413757324
行列サイズ = 4096
処理時間 : memcpy = 0.011600351333618164
処理時間 : kernel = 0.015173029899597169
処理時間 : total  = 0.046537160873413086
